In [1]:
import sys, os, torch, numpy as np
from modules.ig_utils import ig_analysis
from modules.model_utils import model_loss_optimizer_resnet

device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

model, _, _ = model_loss_optimizer_resnet(
    device=device, weight_pos=1.0, lr=1e-3, show_model_summary=False, resnet_depth=50
)
ckpt_path = "/zdisk/users/ext_user_03/01_yschoi/project_01_FVH_detection/02_results/fhv_resnet50_with_cam/best_model.pt"  # ← 너의 체크포인트
ckpt = torch.load(
    ckpt_path,
    map_location=device if isinstance(device, str) else None,
    weights_only=False 
    ) if os.path.exists(ckpt_path) else None

if ckpt and 'model' in ckpt:
    model.load_state_dict(ckpt['model'])
model.eval();

In [ ]:
path_test_data = '/zdisk/users/ext_user_03/01_yschoi/project_01_FVH_detection/02_results/fhv_resnet50_with_cam/test_data.npy'
test_data = np.load(path_test_data)

path_test_label = '/zdisk/users/ext_user_03/01_yschoi/project_01_FVH_detection/02_results/fhv_resnet50_with_cam/test_y_true.npy'
test_label = np.load(path_test_label)

from modules.data_utils import SliceDataset
from torch.utils.data import Dataset, DataLoader

batch_size = 1
num_workers = 0
ds_test  = SliceDataset(test_data, test_label)
dl_test  = DataLoader(ds_test,  batch_size=batch_size, shuffle=False,
                      num_workers=num_workers, pin_memory=False,
                      persistent_workers=False if num_workers == 0 else True)

from modules.ig_utils import ig_on_batch

model.eval()  # 반드시 eval

idx = 0
for images, labels in dl_test:           # images: (B,1,672,672)
    # 배치 한 번에 IG 계산 + PNG 저장
    res_list = ig_on_batch(
        model=model,
        batch_images=images,            # (B,1,H,W) 그대로 전달
        device="cuda:1",
        out_dir="./02_results/fhv_resnet50_with_cam/ig_test", # PNG는 runs/ig_batch_demo/ig_batch/*.png 로 저장
        fname_index=idx,
        ig_steps=64,
        ig_target="pos",                # 1-logit이면 sign(+), multi-logit이면 pos_class_idx 사용
        pos_class_idx=1,                # multi-logit에서 양성 클래스 인덱스가 1이라면
        baseline_mode="zeros",          # 'zeros'|'constant'|'blur'
        use_noise_tunnel=False,         # 필요시 True로
        viz_sign="positive",            # 'positive'|'negative'|'both'|'absolute'
        display_mode="percentile",      # 'percentile'|'zscore'
        save_raw_attr=False
    )
    idx += 1

    # 선택: res_list[i]에는 각 샘플의 attr/overlay/score/delta 등이 들어있음
    # 여기서는 이미지 파일 저장만 쓰는 경우 res_list를 굳이 보관할 필요 없음.

In [ ]:
test_x = test_data[1]
test_y = test_label[1]

# 1채널 → 3채널 복제
sl_3ch = np.repeat(test_x[None, ...], 3, axis=0)  # (3,H,W)
x = torch.from_numpy(sl_3ch)[None, ...]       # (1,3,H,W)
x = x.to(device)

# 아주 간단한 DataLoader 구성 (배치 1개)
from torch.utils.data import TensorDataset, DataLoader
ds = TensorDataset(x)  # 라벨은 없어도 됨
dl_test = DataLoader(ds, batch_size=1, shuffle=False)
len(ds)

1

In [5]:
import torch.nn as nn

# 1) 첫 Conv가 1채널인 모델을 위한 래퍼 (3→1 평균)
class Force1Channel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def forward(self, x):
        # x: (B,C,H,W)
        if x.shape[1] == 3:
            x = x.mean(dim=1, keepdim=True)  # 3→1
        return self.model(x)

# 2) 첫 Conv가 3채널인 모델을 위한 래퍼 (1→3 복제)
class Force3Channel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def forward(self, x):
        if x.shape[1] == 1:
            x = x.repeat(1, 3, 1, 1)  # 1→3
        return self.model(x)

# 3) 네 모델 구조 확인
import torch.nn as nn
first_in = next(m.in_channels for m in model.modules() if isinstance(m, nn.Conv2d))
print("first conv in_channels =", first_in)

# 4) in_channels=1 이라면:
if first_in == 1:
    wrapped = Force1Channel(model).to(device).eval()
else:
    wrapped = Force3Channel(model).to(device).eval()

# 5) IG 실행 (denorm은 1채널 모델이면 None 권장)
ig_analysis(
    out_dir='./tmp_ig_demo/',
    model=wrapped,          # ← 래퍼로 감싼 모델을 전달
    dl_test=dl_test,
    device=device,
    export_ig=True,
    ig_steps=128,
    ig_alpha=0.5,
    ig_cmap="jet",
    ig_target="pos",
    ig_positive_only=True,
    mean_for_denorm=None,   # 1채널이면 None
    std_for_denorm=None,
)

first conv in_channels = 1


In [8]:
from modules.ig_utils import ig_analysis
saved = ig_analysis(
    out_dir="./tmp_ig_demo",
    model=model,            # 학습된 모델 (ckpt 로드 후 .eval())
    dl_test=dl_test,        # 너의 테스트 DataLoader
    device="cuda",
    export_ig=True,
    ig_steps=128,
    ig_alpha=0.5,
    ig_cmap="jet",
    ig_target="pos",        # 2-logit: class 1 / 1-logit: sign(+)
    ig_positive_only=True,
    mean_for_denorm=None,   # 1채널 모델이면 None 권장 (자동 정렬됨)
    std_for_denorm=None,
    save_raw_attr=False,
)
saved[:3]

['./tmp_ig_demo/ig_test/ig_000000.png']